參考範例: [AutoVC實作(Pytorch)](https://ithelp.ithome.com.tw/m/articles/10262975)

Dataset: [VCTK dataset](https://datashare.ed.ac.uk/handle/10283/2950)

# Data Preprocessing

In [2]:
import os 
import pickle
import numpy as np
import soundfile as sf
import librosa
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from librosa.util import normalize
from numpy.random import RandomState


In [3]:
# data dir
rootDir = './DR-VCTK/DR-VCTK/DR-VCTK'

# mel storage dir
targetDir = './spmel'
dirName, subdirList,_ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)
print(subdirList)

Found directory: ./DR-VCTK/DR-VCTK/DR-VCTK
['clean_testset_wav_16k', 'clean_trainset_wav_16k', 'device-recorded_testset_wav_16k', 'device-recorded_trainset_wav_16k']


In [4]:
def mel_gan_handler(x, fft_length=1024, hop_length=256, sr=22050):
    wav = normalize(x)
    p = (fft_length - hop_length) // 2
    wav = np.squeeze(np.pad(wav, (p,p), 'reflect'))
    fft = librosa.stft(
                        wav,
                        n_fft=fft_length,
                        hop_length = hop_length,
                        window = 'hann',
                        center = False
                        )
    
    # 這裡的 abs 是 sqrt(實部**2 + 虛部**2)
    mag = abs(fft)
    mel_basis = mel(sr, 1024, fmin = 0.0, fmax=None, n_mels=80)
    mel_output = np.dot(mel_basis, mag)
    log_mel_spec = np.log10(np.maximum(1e-5,mel_output)).astype(np.float32)
    return log_mel_spec

In [8]:
# VCTK 是 48KHz 我們必須先 resample 到 22.05 KHz
new_rate = 22050

for subdir in sorted(subdirList):
    if not os.path.exists(os.path.join(targetDir, subdir)):
        os.makedirs(os.path.join(targetDir, subdir))
    
    _,_, fileList= next(os.walk(os.path.join(dirName,subdir)))
    
    for fileName in sorted(fileList):
        x, fs = sf.read(os.path.join(dirName,subdir,fileName))
        ##############
        # change sample rate from 48000 -> 22050
        # Since mel_gan use 22050
        ##############
        x = librosa.resample(x, fs, new_rate)
        S = mel_gan_handler(x)
        
        np.save(os.path.join(targetDir, subdir, fileName[:-5]), 
               S.astype(np.float32), allow_pickle=False)
        
    print(f"Done --- {subdir}")

C:\Users\jason\AppData\Local\Temp/ipykernel_17992/1697858801.py:16: FutureWarning: Pass orig_sr=16000, target_sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  x = librosa.resample(x, fs, new_rate)
C:\Users\jason\AppData\Local\Temp/ipykernel_17992/771229521.py:15: FutureWarning: Pass sr=22050, n_fft=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = mel(sr, 1024, fmin = 0.0, fmax=None, n_mels=80)
C:\Users\jason\AppData\Local\Temp/ipykernel_17992/1697858801.py:16: FutureWarning: Pass orig_sr=16000, target_sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  x = librosa.resample(x, fs, new_rate)


Done --- clean_testset_wav_16k
Done --- clean_trainset_wav_16k
Done --- device-recorded_testset_wav_16k
Done --- device-recorded_trainset_wav_16k


# 生成 D_VECTOR

下載 [Pre_train Model](https://drive.google.com/file/d/1ORAeb4DlS_65WDkQN6LHx5dPyCM5PAVV/view) 後定義 D_VECTOR, 這裡用的是 LSTM版的

In [2]:
import torch
import torch.nn as nn

class D_VECTOR(nn.Module):
    """ D_VECTOR speaker embedding."""
    
    def __init__(self, num_layers=3, dim_input=40, dim_cell=256, dim_emb=64):
        super(D_VECTOR, self).__init__()
        self.lstm = nn.LSTM(input_size=dim_input, hidden_size=dim_cell,
                           num_layers=num_layers, batch_first=True)
        self.embedding = nn.Linear(dim_cell, dim_emb)
        
    def forward(self, x):
        self.lstm.flatten_parameters()
        lstm_out, _ = self.lstm(x)
        embeds = self.embedding(lstm_out[:,-1,:])
        norm = embeds.norm(p=2, dim=-1, keepdim=True)
        embeds_normalized = embeds.div(norm)
        return embeds_normalized

接著要 Load_state_dict, 注意 num_uttrs 這個參數

In [3]:
import os
import pickle
# from model_bl import D_VECTOR
from collections import OrderedDict
import numpy as np
import torch

C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cpu()
c_checkpoint = torch.load('3000000-BL.ckpt',map_location=torch.device('cpu'))
new_state_dict = OrderedDict()

for key, val in c_checkpoint['model_b'].items():
    new_key = key[7:]
    new_state_dict[new_key] = val
    
C.load_state_dict(new_state_dict)

# 指的是說一個語者說了幾種不同內容的話, 讓資料的數量盡量一樣, 內容可以不一樣
num_uttrs = 68
len_crop = 176

# Directory containing mel-spectrograms
rootDir = './spmel'
dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)

def pad_along_axis(array: np.ndarray, target_length: int, axis: int=0):
    pad_size = target_length - array.shape[axis]
    
    if pad_size <= 0:
        return array
    npad = [(0,0)] * array.ndim
    npad[axis] = (0,pad_size)
    
    return np.pad(array, pad_size)

speakers = []
for speaker in sorted(subdirList[1:]):
    print('Processing speaker: %s' % speaker)
    utterances = []
    utterances.append(speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))
    fileList = fileList[:num_uttrs]
    
    # make speaker embedding
    assert len(fileList) >= num_uttrs
    idx_uttrs = np.random.choice(len(fileList), size=num_uttrs, replace=False)
    embs = []

    for i in range(num_uttrs):
        tmp = np.load(os.path.join(dirName, speaker, fileList[idx_uttrs[i]]),allow_pickle=True)
        
        # pad if the current one is too short
        if tmp.shape[0] <= len_crop:
            pad =int(len_crop - tmp.shape[0])
            tmp = pad_along_axis(tmp,pad)
            melsp = torch.from_numpy(tmp[np.newaxis,:, :])
        else:
            left = np.random.randint(0, tmp.shape[0]-len_crop)
            melsp = torch.from_numpy(tmp[np.newaxis, left:left+len_crop, :])
        
        emb = C(melsp)
        embs.append(emb,detach().squeeze.cpu().numpy())
        
utterance.append(np.mean(embs,axis=0))
for fileName in sorted(fileList):
    utterances.append(os.path.join(speaker,fileName))
speakers.append(utterance)

with open(os.path.join(rootDir,'train.pkl'),'wb') as handle:
    pickle.dump(speakers,handle)



Found directory: ./spmel
Processing speaker: clean_trainset_wav_16k


RuntimeError: input.size(-1) must be equal to input_size. Expected 80, got 342

# 製作 DataLoader

In [1]:
from torch.utils import data
import torch
import numpy as np
import pickle
import os
from multiprocessing import Process, Manager

class Utterances(data.Dataset):
    """Data class for the Utterances dataset"""
    
    def __init__(self,root_dir,len_crop):
        """Initialize and preprocess the Utterances dataset."""
        
        self.root_dir = root_dir
        self.len_crop = len_crop
        self.step = 10
        
        metaname = os.path.join(self.root_dir, "train.pkl")
        meta = pickle.load(open(metaname,'rb'))
        
        """Load data using multiprocessing"""
        manager = Manager()
        meta = manager.list(meta)
        dataset = manager.list(len(meta)*[None])
        processes = []
        
        for i in range(0,len(meta), self.step):
            p = Process(target=self.load_data,
                        args =(meta[i:i+self.step],dataset,i))
            p.start()
            processes.append(p)
            
        for p in processes:
            p.join()
            
        self.train_dataset = list(dataset)
        self.num_tokens = len(self.train_dataset)
        
        print('Finished loading the dataset...')
        
    def load_data(self,submeta,dataset,idx_offset):
        for k, sbmt in enumerate(submeta):
            uttrs = len(sbmt)*[None]
            for j, tmp in enumerate(sbmt):
                if j<2: # fill in speaker id and embedding
                    uttrs[j] = tmp
                else: # load the mel-spectrograms
                    uttrs[j] = np.load(os.path.join(self.root_dir,tmp))
            dataset[idx_offset+k] = uttrs
            
    def __getitem__(self,index):
        #pick a random speaker
        dataset = self.train_dataset
        list_uttrs =dataset[index]
        emb_org = list_uttrs[1]
        
        # pick random uttr with random crop
        a = np.random.randint(2,len(list_uttrs))
        tmp = list_uttrs[a]
        
        if tmp.shape[0]<self.len_crop:
            len_pad = self.len_crop - tmp.shape[0]
            uttr = np.pad(tmp, ((0,len_pad),(0,0)),'constant')
        elif tmp.shape[0]> self.len_crop:
            left = np.random.randint(tmp.shape[0]-self.len_crop)
            uttr = tmp[left:left+self.len_crop, :]
        else:
            uttr = tmp
            
        return uttr, emb_org
    
    def __len__(self):
        """Return the number of speakers."""
        return self.num_tokens
    
def get_loader(root_dir,batch_size=2, len_crop=176,num_workers=0):
    """Build and return a data loader."""
    
    dataset = Uttrances(root_dir,len_crop)
    
    worker_init_fn = lambda x: np.random.seed((torch.initial_seed()) % (2**32))
    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size = batch_size,
                                  shuffle=True,
                                  num_workers = num_workers,
                                  drop_last=True,
                                  worker_init_fn = worker_init_fn)
    return data_loader
                                    

使用時只需要

In [19]:
vcc_loader = get_loader('./spmel',BATCH_SIZE,LEN_CROP)

for j in range(step):
    try:
        x_real, emb_org = next(data_iter)
    except:
        data_iter = iter(vcc_loader)
        x_real, emb_org = next(data_iter)
        '''
        train model here
        '''

NameError: name 'BATCH_SIZE' is not defined

# AutoVC
先參考官網  [model_vc.py](https://github.com/auspicious3000/autovc/blob/master/model_vc.py)

# 製作 Solver

把官網 [solver_encoder.py](https://github.com/auspicious3000/autovc/blob/master/solver_encoder.py)

In [21]:
# 原本會造成 shape miss-match 導致收斂過快無法學習 
g_loss_id = F.mse_loss(x_real, x_identic.squeeze())  
g_loss_id_psnt = F.mse_loss(x_real, x_identic_psnt.squeeze())   

NameError: name 'F' is not defined

# Train

In [ ]:
import torch
from solver_encoder import Solver
from data_loader import get_loader

class Config:
    def __init__(self):
        self.data_dir = './spmel'
        self.batch_size = 2
        self.len_crop = 176
        self.lambda_cd =1
        self.dim_neck = 44
        self.dim_emb = 256
        self.dim_pre = 512
        self.freq = 22
        self.num_iters = 1000000
        slf.log_step = 10

config = Config()
vcc_loader = get_loader(config.data_dir, config.batch_size, config.len_crop)
solver = Solver(vcc_loader, config)
solver.train()
torch.save(solver.G.state_dict(),'autovc')

# Inference

In [ ]:
import IPython.display as ipd
import pickle
import torch
import numpy as np
from model_vc import Generator

device = 'cuda:0'
G = Generator(32,256,512,22).eval().to(device)
G.load_state_dict(torch.load('autovc'))
metadata = pickle.load(open('spmel/train.pkl','rb'))

source = 0
target = 3

# 自定義 source
uttr - np.load(f"spmel/.../... .npy")
#(1,256)
emb_org = torch.from_numpy(np.expand_dims(metadata[source][1],axis=0)).to(device)
#(1,256)
emb_trg = torch.from_numpy(np.expand_dims(metadata[target][1],axis=0)).to(device)

#(1,178,80)
uttr = torch.from_numpy(np.expand_dims(uttr,axis=0)).to(device)
uttr_trg = None

with torch.no_grad():
    _, x_identic_psnt,_ = G(uttr,emb_org, emb_trg)

#(176,80)
uttr_trg = x_identic_psnt[0,0,L,L].cpy().numpy()

# To Waveform
from interface import *
vocoder = MelVocoder()
audio = np.squeeze(vocoder.inverse(torch.from_numpy(np.expand_dims(uttr_trg.T,axis=0))).cpu().numpy())
ipd.Audio(audio,rate=22050)

# Pytorch with CUDA Problem

AssertionError: Torch not compiled with CUDA enabled

In [14]:

print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cpu
False


In [8]:
X_train = torch.FloatTensor([0.,1.,2.])

In [9]:
X_train= X_train.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [28]:
torch.cuda.current_device()

AssertionError: Torch not compiled with CUDA enabled

In [15]:
torch.cuda.device_count()

0

https://www.datasciencelearner.com/assertionerror-torch-not-compiled-with-cuda-enabled-fix/#:~:text=AssertionError%3A%20torch%20not%20compiled%20with%20Cuda%20enabled%20error%20occurs%20because,false%20or%20removing%20the%20same.

https://pytorch.org/

[Why torch.cuda.is_available() always False](https://stackoverflow.com/questions/60987997/why-torch-cuda-is-available-returns-false-even-after-installing-pytorch-with)


[淺談Pytorch與 Torch關係](https://oldpan.me/archives/pytorch-torch-relation)
